<h1><b>Statistique en Bioinformatique : </b> TME3 </h1>
<br>
L’objectif de ce TME est:
<br>
<ul>
<li> objectif 1: comprendre les principaux etaphes de BLAST et PSIBLAST, </li>
<li> objectif 2: comprendre et comparer les algoritime de l'alignement progressive </li>
</ul>
<br>
<div class="alert alert-warning" role="alert" style="margin: 10px">
<p><b>Soumission</b></p>
<ul>
<li>Soumettre sur moodle </li>
</ul>
</div>


Nom etudiant 1 : Garcia Enzo
<br>
Nom etudiant 2 : Tantouch Maher
<br>

<h3>Section 1: BLAST</h3>

<b> Exercice 1 </b>: À partir de l'application BLAST, disponible via le site de <a href="http://blast.ncbi.nlm.nih.gov/">NCBI</a>, cliquez sur protein Blast. Faites un BLAST de la séquence fasta/seq1.fasta. Expliquez le résultat graphique, la couleur de chaque hit et expliquez le tableau de résultat qu'est-ce que c'est chaque colonne?
<br>


<font color="red"> Réponse : </font>
<font color="blue">
Le résultat graphique illustre l'alignement de la séquence 1 avec les séquences de la base de données. La couleur rouge correspond aux parties de séquences qui ont un score d'alignement élevé supérieur à 200 (identité élevée et faibles gaps) ; la couleur rose : score d'alignement situé entre 80 et 200 (alignement intermédiaire) ; la couleur vert : score d'alignement entre 50 et 80 (faible identité/plus de gaps).

Observations : on remarque que les scores d'alignement plus faibles sont situés aux extrémités de la séquence d'intérêt et que les plus 

Dans le tableau de résultats les colonnes sont (droite à gauche) : 
- "Description" : description et nom de la séquence - provenant de la base de donnée - alignée.
- "Scientific names": la référence taxonomique de l'espèce étudiée
- "Max Score": le score maximum pour l'alignement entre la séquence query et la séquence cible. Plus il est élevé et plus l'alignement est meilleur.
- "Total Score": Somme des score de tous les alignements pour la même séquence cible.
- "Query Cover": Pourcentage de la séquence query couvert par l'alignement.
- "E-value": Probabilité que l'alignement soit dû au hasard. Une valeur très petite indique un alignement significatif.
- "Accession length": longueur de la séquence cible.
- "Accession": Identifiant de la séquence cible dans la base de donnée.


</font>

<b>Exercice 2</b>: On s'intéresse à présent au gène FHIT chez l'homme (fragile histidine triad protein, identifiant P49789, télécharger du site  <a href="http://www.uniprot.org/"> Uniprot </a>). La protéine de ce gène est structurellement proche des galactose-1-phosphate uridylyltransferases. Faites un BLAST de ce gène contre la base SWISSPROT. Que constatez-vous?

<font color="red"> Réponse : </font>
<font color="blue">
On constate que FHIT humain montre une identité forte (>90%) avec FHIT des autres espèces, ce qui témoigne d'une grande conservation évolutive. La structure proche entre les deux protéines explique leur similarité dans les alignements. Cette similarité structurelle citée dans l'enoncé pourrait donc s'expliquer par une origine évolutive commune.
</font>

<b>Exercice 3</b>: Implémenter l'étape 0 du algorithme BLAST (indexer la base de données), utilise w=5 et les séquences du fichier database.fasta.

In [146]:
import numpy as np

#variable initiation
database = 'database.fasta'
seqTestBlast = 'test.fasta'
input_blosum_f = 'BLOSUM62.txt'
aa = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', 'X']

#parameters
q = 21
gap = -5
w = 5

#For a simple test use:
database = 'databaseToy.fasta'
seqTestBlast = 'testToy.fasta'
w=3

In [156]:

def etape0(fileName, w):
    """
    Indexation de la base de données
    Entrée1 fileName: nom du fichier FASTA
    Entrée2 w: taille du mot
    Sortie1 index: dictionnaire contenant les mots indexés
    Sortie2 dicoSeq: dictionnaire contenant toutes les séquences de la base
    """
    index = {}
    dicoSeq = {}

    # Lecture du fichier FASTA
    with open(fileName, 'r') as file:
        seq_id = ""
        sequence = ""
        
        for line in file:
            if line.startswith(">"):  # Identifiant de séquence
                if seq_id:  # Enregistrer la séquence précédente
                    dicoSeq[seq_id] = sequence.upper()  # Conversion en majuscules
                seq_id = line.strip()[1:]  # Enlever ">"
                sequence = ""
            else:
                sequence += line.strip()  # Conserver la casse d'origine
                
        if seq_id:  # Ajouter la dernière séquence
            dicoSeq[seq_id] = sequence.upper()  # Conversion en majuscules

    # Indexation des mots en majuscules
    for seq_id, sequence in dicoSeq.items():
        for i in range(len(sequence) - w + 1):
            word = sequence[i:i + w].upper()  # Conversion en majuscules
            if word not in index:
                index[word] = []
            index[word].append((seq_id, i))

    return index, dicoSeq

# Exécution de l'étape 0
index, dicoSeq = etape0(database, w)

# Affichage des résultats
print(index)
print(dicoSeq)
#{'AGH': [('s1', 0)], 'CGG': [('s3', 0)], 'GGH': [('s2', 1)]}
#{'s1': 'AGHR', 's2': 'CGGHR', 's3': 'CGGH'}

{'AGH': [('s1', 0)], 'GHR': [('s1', 1), ('s2', 2)], 'CGG': [('s2', 0), ('s3', 0)], 'GGH': [('s2', 1), ('s3', 1)], 'GHA': [('s3', 2)]}
{'s1': 'AGHR', 's2': 'CGGHR', 's3': 'CGGHA'}


<b>Exercice 4</b>: Chercher la séquence query test.fasta dans la base indexé construite précédemment, entendez l'alignement pour avoir un alignement sans gap et de score > 12. Retournez les quatre meilleurs alignements.


In [158]:
#read Blosum
def readBlosum(nameFile):
    """
    Read substitution matrix file
    Input1 fileName : BLOSUM file name
    Output1 Blosum: a dictionary containing BLOSUM matrix, key is a pair of amino acids (alphabetic order) and value is the substitution value.
    """
    Blosum = {}
    with open(nameFile, 'r') as f:
        lines = [line.strip() for line in f.readlines() if not line.startswith("#") and line.strip()]
        aa_list = lines[0].split()

        for line in lines[1:]:
            values = line.split()
            row_aa = values[0]
            scores = list(map(int, values[1:]))

            for col_aa, score in zip(aa_list, scores):
                Blosum[(row_aa, col_aa)] = score
                Blosum[(col_aa, row_aa)] = score  # Assurer la symétrie

    return Blosum


blosum = readBlosum(input_blosum_f)
print (blosum["A", "A"])
#4

4


In [160]:
def extendAln(pos1, pos2, sequence1, sequence2, matriceBlosum, scoremotif, w, ThrScore):
    """
    Extend alignment left/right by keeping score above a threshold
    Input1 pos1 : position in the first sequence (Subject)
    Input2 pos2 : position in the second sequence (query)
    Input3 sequence1 : first sequence (Subject)
    Input4 sequence2 : second sequence (query)
    Input5 matriceBlosum: a dictionary containing BLOSUM matrix
    Input6 scoremotif: initial score
    Input7 w: word size
    Input8 ThrScore: threshold
    Output 1 finalScore: final score
    Output 2 algLength: alignment length
    Output 3 alnSeqs: Tuple containing aligned sequences
    """

    # Convertir les séquences en majuscules pour éviter les erreurs de capitalisation
    sequence1 = sequence1.upper()
    sequence2 = sequence2.upper()

    # Initialiser les variables
    finalScore = scoremotif
    algLength = w  # Initialement, la longueur de l'alignement est la taille du mot
    alnSeqs = (sequence1[pos1:pos1 + w], sequence2[pos2:pos2 + w])  # Motif initial

    # Extension à gauche
    while pos1 > 0 and pos2 > 0:
        pos1 -= 1
        pos2 -= 1
        score = matriceBlosum[(sequence1[pos1], sequence2[pos2])]
        finalScore += score
        if finalScore < ThrScore:
            break
        algLength += 1
        alnSeqs = (sequence1[pos1:pos1 + algLength], sequence2[pos2:pos2 + algLength])

    # Extension à droite
    while pos1 + algLength < len(sequence1) and pos2 + algLength < len(sequence2):
        score = matriceBlosum[(sequence1[pos1 + algLength], sequence2[pos2 + algLength])]
        finalScore += score
        if finalScore < ThrScore:
            break
        algLength += 1
        alnSeqs = (sequence1[pos1:pos1 + algLength], sequence2[pos2:pos2 + algLength])

    return [finalScore, algLength, alnSeqs]

res = extendAln(1, 1, dicoSeq['s2'], 'AGGHV', blosum, 20, 3, 18)
print (res)
#[20, 4, ('CGGH', 'AGGH')]
res = extendAln(1, 1, dicoSeq['s2'], 'AGGHV', blosum, 20, 3, 13)
print (res)
#[17, 5, ('CGGHR', 'AGGHV')]


[17, 4, ('CGGH', 'AGGH')]
[17, 5, ('CGGHR', 'AGGHV')]


In [164]:
def getWordsQuery(w, query, indexB):
    """
    Looking for indexed words in the query
    Input1 w: word size
    Input2 query: sequence
    Input3 indexB: indexed database
    Output1 foundWords: Tuple containing found words, position in query, list of pairs (Subject id, Subject position)
    """
    foundWords = []
    
    # Parcourir la séquence de la requête pour extraire des motifs de taille w
    for i in range(len(query) - w + 1):
        word = query[i:i + w].upper()  # Récupère un mot et le convertit en majuscules
        if word in indexB:  # Si le mot est présent dans l'index
            # Ajouter le mot, la position et les positions de l'index
            foundWords.append((word, i, indexB[word]))
    
    return foundWords

foundWords = getWordsQuery(w,  'AGGHV', index)
print (foundWords)
#sortie attendue : [('GGH', 1, [('s2', 1)])]


[('GGH', 1, [('s2', 1), ('s3', 1)])]


In [170]:
#Implement BLAST algorithm

def Blast(foundWords, indexedDB, Query, dicoSeq, T, w, matriceBlosum):
    """
    Implements the BLAST algorithm
    Input1 foundWords: Tuple containing found words, position in query, list of pairs (Subject id, Subject position)
    Input2 indexedDB: indexed database
    Input3 Query: Query sequence
    Input4 dicoSeq: dictionary containing database sequences
    Input5 T: Threshold score
    Input6 w: word size
    Input7 matriceBlosum: BLOSUM matrix
    Output1 alignedSeq: Tuple of aligned sequences
    """
    alignedSeq = []

    # Fonction pour calculer le score d'alignement
    def alignment_score(seq1, seq2, matriceBlosum):
        score = 0
        for i in range(min(len(seq1), len(seq2))):
            score += matriceBlosum.get((seq1[i], seq2[i]), 0)  # Si la paire de bases n'est pas dans la matrice, score = 0
        return score

    # Filtrer les mots trouvés et effectuer l'alignement
    for word, pos_query, subjects in foundWords:
        for subject_id, pos_subject in subjects:
            db_sequence = dicoSeq[subject_id]
            
            # Extraire les sous-séquences de longueur w à partir de la base de données et de la requête
            query_subseq = Query[pos_query:pos_query + w]
            db_subseq = db_sequence[pos_subject:pos_subject + w]
            
            # Calculer le score d'alignement
            score = alignment_score(query_subseq, db_subseq, matriceBlosum)
            
            # Si le score est supérieur au seuil, ajouter l'alignement à la liste des séquences alignées
            if score >= T:
                alignedSeq.append((query_subseq, db_subseq))

    return alignedSeq

ThrScore = 12
alignedSeq = Blast(foundWords, index, "AGGHV", dicoSeq, ThrScore, w, blosum)  #j'ai changé l'ordre des arguments avec paramètre matriceBlosum donné à la fin
print(alignedSeq)



[('GGH', 'GGH'), ('GGH', 'GGH')]


<h3>Section 2: PSI-BLAST</h3>
On considère à nouveau le gène FHIT chez l'homme.
<BR>
<b> Exercice 1 </b>:
Faites un PSI-BLAST de ce gène contre la base SWISS-PROT. Utilisez un seuil pour l’évalue de 1e-5.
Que constatez-vous lors de la 1ère itération concernant les protéines de la famille des galactose-1-phosphate uridylyltransferases?


<font color="red"> Réponse : </font>
<font color="blue">
Au cours de la première itération, on peut voir que PSI-BLAST génère un profil de position qui est spécifique de la séquence de départ. Ce qui permet de mieux identifier des protéines similaires et par conséquent d'affiner le profil pour les itérations suivantes.
</font>

<b> Exercice 2 </b>:
Que constatez-vous lors de la 2eme itération? Lors de la 5eme itération?


<font color="red"> Réponse : </font>
<font color="blue">
A la 2ème itération l'affinement de la recherche permet l'ajout de nouveux homolgues et une meilleur couverture des équences. Cependant le pourcentage d'identité diminue. 
Lors de la 5ème itération, on a un affinement maximal avec détection de nouveaux homologues - qui ont toutefois des similarités plus faibles. Dans le même temps, on a des résultats plus spécifiques et un affinement maximal du profil de séquence.
</font>

<b> Exercice 3 </b>:
À partir de quelle itération peut-on considérer qu'il y a eu une convergence de l'algorithme PSI- BLAST? Pour quoi ?

<font color="red"> Réponse : </font>
<font color="blue">
On peut considérer avoir atteint une convergence de l'algorithme dès lors que ce sont les mêmes séquences/motifs qui sont identifiés à chaque itération et que pas ou peu de nouvelles séquences significatives sont ajoutées aux alignements.
</font>

<h3>Section 3: Alignement Multiple  </h3>


<b> Exercice 1 </b>:
Accédez à l'application <a href="http://www.ebi.ac.uk/Tools/msa/clustalo/">Clustal omega</a> et copiez les 7 séquences du fichier 7sequences.fasta. Expliquez, dans quel ordre les séquences ont été alignées.

<font color="red"> Réponse : </font>
<font color="blue">
Lors d'un alignement multiple, l'alignement est progressif c'est à dire qu'il commence par aligner les deux séquences les plus similaires puis les autres séquences sont alignées progressivement avec cet alignement initial.
Explications : sp|P69905|HBA_HUMAN et sp|P02062|HBB_HORSE ont été alignés en premier car elles sont très proches. Suivi de sp|P68871|HBB_HUMAN, sp|P02062|HBB_HORSE. Ensuite, on a sp|P02240|LGB2_LUPLU et sp|P02185|MYG_PHYMC qui , plus éloignés, sont alignées ensemble. Enfin la séquence sp|P02208|GLB5_PETMA est inetgrée en fin car la moins similaires.
Arbre phylogénétique obtenu : 
(
sp|P02240|LGB2_LUPLU:0.45433,
sp|P02185|MYG_PHYMC:0.37675,
(
sp|P02208|GLB5_PETMA:0.35874,
(
(
sp|P69905|HBA_HUMAN:0.05936,
sp|P01958|HBA_HORSE:0.06035)
:0.21277,
(
sp|P68871|HBB_HUMAN:0.08313,
sp|P02062|HBB_HORSE:0.08125)
:0.21840)
:0.06156)
:0.02342);


</font>

<b> Exercice 2 (OPTIONAL)  </b>:
Faite une fonction pour calculer le score d’un alignement multiple. Le score de chaque position est la somme des tous les scores obtenus par BLOSUM62, et le score final est la somme des scores de chaque position divisée par la taille du alignement.

In [205]:
#Read multi fasta
def extraireAllFastaMulti(fileName):
    """
    Read a FASTA file with several sequences
    input1 fileName: file name
    output1 IdSeq: list of sequences IDs
    output2 Seqs: list of sequences
    """
    Seqs = []
    IdSeq = []

    # Lire le fichier FASTA
    with open(fileName, 'r') as file:
        seq_id = ""
        sequence = ""
        
        for line in file:
            if line.startswith(">"):  # Identifiant de séquence
                if seq_id:  # Enregistrer la séquence précédente
                    Seqs.append(sequence)
                    IdSeq.append(seq_id)
                seq_id = line.strip()[1:]  # Enlever ">"
                sequence = ""
            else:
                sequence += line.strip()
        
        # Ajouter la dernière séquence
        if seq_id:
            Seqs.append(sequence)
            IdSeq.append(seq_id)

    return IdSeq, Seqs

In [223]:
def SP_score(SeqsM, blosum, gap):
    """
    Compute MSA score based on a substitution matrix and gap penalty.
    input1 SeqsM: list of aligned sequences
    input2 blosum: substitution matrix
    input3 gap: gap penalty
    output1 score: MSA score
    """
    
    # Vérification de la longueur des séquences
    max_len = max(len(seq) for seq in SeqsM)  # Trouver la longueur maximale des séquences
    SeqsM = [seq.ljust(max_len, '-') for seq in SeqsM]  # Compléter les séquences plus courtes avec des gaps

    score = 0
    # Transposer les séquences pour itérer sur les colonnes
    for i in range(len(SeqsM[0])):  # Parcours des colonnes
        column = [seq[i] for seq in SeqsM]  # Colonne de la position i dans chaque séquence
        for j in range(len(column)):
            for k in range(j + 1, len(column)):  # Comparer chaque paire de séquences dans la colonne
                base1, base2 = column[j], column[k]
                # Si c'est un gap, ajouter la pénalité de gap
                if base1 == '-' or base2 == '-':
                    score += gap
                elif base1 != base2:  # Sinon, utiliser la matrice de substitution
                    score += blosum.get((base1, base2), 0)  # Si le couple n'existe pas dans la matrice, ajouter 0
                else:  # Si les bases sont identiques, aucun changement de score
                    score += 0
    return score
SeqsM = ['ACG', 'A-G', 'ACD']
score = SP_score(SeqsM, blosum, gap)
print (score)
#5.0

-6


<b> Exercice 3 </b>: Aligner les séquences du fichiers algnMult2.fasta avec  <a href="http://www.ebi.ac.uk/Tools/msa/clustalo/">Clustal omega</a>. Utiliser la fonction développé precedement  pour calculer le score du alignement. Quelle est la séquence la plus divergente?

<font color="red"> Réponse : </font>
<font color="blue">
La séquence la plus divergente semble être Sequence4, car elle contient de nombreux gaps 
</font>

In [227]:
# Charger les séquences du fichier FASTA
IdSeq, Seqs = extraireAllFastaMulti("algnMult.fasta")

# Lire la matrice BLOSUM62
input_blosum_f = "blosum62.txt"  # Remplace par le chemin correct de ton fichier BLOSUM
blosum = readBlosum(input_blosum_f)

# Définir la pénalité de gap
gap = -2

# Calculer le score de l'alignement
score = SP_score(Seqs, blosum, gap)
print(f"Le score de l'alignement est: {score}")


Le score de l'alignement est: -200
